In [4]:
from anchor_wrap import anchor_call

import math
from options import Options
import os
import random
import shutil
import subprocess
import sys
from xgbooster import XGBooster
import resource




num = 50
verbose = 1

with open("datasets.list", 'r') as fp:
    datasets = [tuple(line.strip().split(' ')) for line in fp.readlines() if line]


root="results"
if not os.path.exists(root):
    os.makedirs(root)  
if not os.path.exists(f'{root}/anchor'):
    os.makedirs(f'{root}/anchor')  


# initializing the seed
random.seed(1234)

options = Options(f'./xreason.py  -q  -x \'inst\' somefile'.split())
    

for data, depth in datasets:
    print(f'processing {data}...')

    # reading and shuffling the instances
    with open(data, 'r') as fp:
        insts = [line.strip().rsplit(',', 1)[0] for line in fp.readlines()[1:]]
        insts = list(set(insts))
        random.shuffle(insts)

        nof_insts = min(200, len(insts))
        print(f'considering {nof_insts} instances')
   
        
    base = os.path.splitext(os.path.basename(data))[0]
    mfile = f'../models/{base}/{base}_nbestim_{num}_maxdepth_{depth}_testsplit_0.2.mod.pkl'
    
    ''''
    with open(f'../logs/smt/{base}.log', 'r') as fp:
        lines = fp.readlines()
        insts = [line[3:].strip() for line in lines if line.startswith('i:')]
        nof_insts = len(insts)
        print(f'considering {nof_insts} instances')
   '''     

    log = open(f'{root}/anchor/{base}.log', 'w')

    # creating booster objects
    xgb = XGBooster(options, from_model=mfile)

    atimes = []
            
    for i, inst in enumerate(insts):

        # processing the instance
        options.explain = [float(v.strip()) for v in inst.split(',')]
            
        timer = resource.getrusage(resource.RUSAGE_CHILDREN).ru_utime + \
                resource.getrusage(resource.RUSAGE_SELF).ru_utime            

        expl1 = xgb.explain(options.explain, use_anchor=anchor_call)
            
        timer = resource.getrusage(resource.RUSAGE_CHILDREN).ru_utime + \
                resource.getrusage(resource.RUSAGE_SELF).ru_utime - timer            

        print(f'i: {inst}', file=log)
        print(f's: {len(expl1)}', file=log)
        print(f't: {timer:.3f}', file=log)
        print('', file=log)

        atimes.append(timer)

        log.flush()
        sys.stdout.flush()

    ##################
    print(f"max time: {max(atimes):.2f}", file=log)
    print(f"min time: {min(atimes):.2f}", file=log)
    print(f"avg time: {sum(atimes)/len(atimes):.2f}", file=log)
    print("", file=log)

    #################
    log.close()
    log.close()

    print('done')



processing ../bench/ann-thyroid/ann-thyroid.csv...
considering 200 instances
considering 200 instances
loading model from  ../models/ann-thyroid/ann-thyroid_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl
loading data from  ../models/ann-thyroid/ann-thyroid_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl.splitdata.pkl
done
processing ../bench/appendicitis/appendicitis.csv...
considering 106 instances
considering 106 instances
loading model from  ../models/appendicitis/appendicitis_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl
loading data from  ../models/appendicitis/appendicitis_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl.splitdata.pkl
done
processing ../bench/biodegradation/biodegradation.csv...
considering 200 instances
considering 200 instances
loading model from  ../models/biodegradation/biodegradation_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl
loading data from  ../models/biodegradation/biodegradation_nbestim_50_maxdepth_3_testsplit_0.2.mod.pkl.splitdata.pkl
done
processing ../bench/divor